In [1]:
import evadb
import shutil
import pandas as pd

In [2]:
shutil.rmtree('./evadb_data')

try:
    cursor = evadb.connect().cursor()
    cursor.query("""
        CREATE DATABASE mydb WITH ENGINE = 'sqlite', PARAMETERS = {
        "database": ":memory:"
    };""").execute()
except:
    pass

In [34]:


# drop all tables
cursor.query("""
    DROP TABLE IF EXISTS tasks;
""").execute()
# cursor.query("""
#     DROP TABLE IF EXISTS members;
# """).df()

# Create tables
cursor.query("""
    CREATE TABLE tasks (
        id INTEGER,
        name TEXT(30),
        description	TEXT(100)
    );
""").execute()


In [39]:
# list tables
x = cursor.query("""
use mydb {
   SELECT 
    name
FROM 
    sqlite_schema
WHERE 
    type ='table' AND 
    name NOT LIKE 'sqlite_%'
};
""").df()

print(x)

Empty DataFrame
Columns: []
Index: []


In [43]:
# create tasks

# y = cursor.query("""
#  INSERT INTO tasks (id, name, description) VALUES (3, 'test name 2', 'test description');
#     """).df()
# print(y)


# drop all data in tasks

# read json using pd
json_data = pd.read_json('issues.json')
print(json_data.head())

# insert each row into table
for index, row in json_data.iterrows():
    # replace all ' with '' to escape
    cursor.query("""
    INSERT INTO tasks (id, name, description) VALUES ({}, '{}', '{}');
    """.format(index, row['contributor'], row['description'][0:1])).execute()

                                         description contributor
0  For now, if open's pr adds or changes long int...     xzdandy
1  Check if the query plan uses VectorIndexScan f...    jiashenC
2  I'm planning to add a new CONCAT operator to E...  pchunduri6
3  When training models, tuning the parameters ar...     xzdandy
4  Right now, the binder_context is not getting u...   gaurav274


In [45]:
x = cursor.table('tasks').select("*").df()
print(x.head())

   tasks._row_id  tasks.id  tasks.name  \
0              1         0     xzdandy   
1              2         1    jiashenC   
2              3         2  pchunduri6   
3              4         3     xzdandy   
4              5         0     xzdandy   

                                   tasks.description  
0  For now. if opens pr adds or changes long inte...  
1  Check if the query plan uses VectorIndexScan f...  
2  Im planning to add a new CONCAT operator to Ev...  
3  When training models. tuning the parameters ar...  
4                                         For now, i  


c:\Users\Rambe\anaconda3\envs\evadb\Lib\site-packages\evadb\models\storage\batch.py:271: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  new_frames = pd.concat(frames, axis=1, copy=False, ignore_index=False).fillna(


In [46]:
x = cursor.query("""
    EXPLAIN SELECT * FROM tasks;
""").execute()

print(x)

                                                                 0
0  |__ ProjectPlan\n    |__ SeqScanPlan\n        |__ StoragePlan\n
